In [42]:
#SQAD Dataset
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')
questions = pd.read_json('questions.json')
count, total = 0,0

for i in range(len(questions)):
    #keys: {title, paragraphs}
    context = (questions['data'][i]['paragraphs'][0]['context'])
    print(context)
    for j in range(len(questions['data'][i]['paragraphs'][0]['qas'])):
        question = (questions['data'][i]['paragraphs'][0]['qas'][j]['question'])
        answer = (questions['data'][i]['paragraphs'][0]['qas'][j]['answers'][0]['text'])
        print("Input Question: ", question)
        inputs = tokenizer(question, context, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        answer_start_index = torch.argmax(outputs.start_logits)
        answer_end_index = torch.argmax(outputs.end_logits)
        predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
        print("Model Response: ", tokenizer.decode(predict_answer_tokens))
        print("Actual Answer: ", answer)
        for word in (answer.lower().split(" ")):
            if (word in tokenizer.decode(predict_answer_tokens).lower().split(" ")):
                count += 1
                break
        total += 1
        print( )
        
print("Accuracy: ", count/total) 

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Input Question:  Which NFL team represented the AFC at Super Bowl 50?
Model Response:  denver broncos
Actual Answer:  Denver Broncos

Input Question:  Which NFL team represented the NFC at Super Bowl 50?
Model Response:  car

In [60]:
#HuggingFace SQUAD_V2 Dataset
from datasets import load_dataset

dataset = load_dataset("squad_v2")

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')
count, total = 0,0

for validation in dataset['validation']:
    context = validation['context']
    question = validation['question']
    answers = validation['answers']['text']

    if answers.empty:
        continue

    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        try:
            outputs = model(**inputs)
        except:
            print('Failure')
            continue
    answer_start_index = torch.argmax(outputs.start_logits)
    answer_end_index = torch.argmax(outputs.end_logits)
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

    model_response = tokenizer.decode(predict_answer_tokens)
    print("Model Response: ", model_response)
    print("Actual Answer: ", answers)

    if model_response.lower() in [x.lower() for x in answers]:
        count += 1
    total += 1
    print( )
print("Accuracy: ", count/total)

Found cached dataset squad_v2 (C:/Users/nsiyu/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Model Response:  france
Actual Answer:  ['France', 'France', 'France', 'France']

Model Response:  10th and 11th centuries
Actual Answer:  ['10th and 11th centuries', 'in the 10th and 11th centuries', '10th and 11th centuries', '10th and 11th centuries']

Model Response:  denmark, iceland and norway
Actual Answer:  ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway']

Model Response:  rollo
Actual Answer:  ['Rollo', 'Rollo', 'Rollo', 'Rollo']

Model Response:  10th
Actual Answer:  ['10th century', 'the first half of the 10th century', '10th', '10th']

Model Response:  the normans
Actual Answer:  []

Model Response:  normandy
Actual Answer:  []

Model Response:  west francia
Actual Answer:  []

Model Response:  first half of the 10th century
Actual Answer:  []

Model Response:  william the conqueror
Actual Answer:  ['William the Conqueror', 'William the Conqueror', 'William the Conqueror']

Model Response:  richard 

In [51]:
!git lfs install
!git clone https://huggingface.co/datasets/NLPC-UOM/Travel-Dataset-5000

Git LFS initialized.


Cloning into 'Travel-Dataset-5000'...
